In [1]:
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import numpy as np
import glob
from sklearn import metrics
from scipy.stats import wilcoxon
import pandas as pd

In [2]:
wavenet_preds_dir = "preds/test_preds_wavenet/preds_test_WaveNet_"
wavenet_targets_dir = "targets/targets_wavenet/targets_test_WaveNet_"
wavenet_labels_dir = "labels/test_labels_wavenet/label_list_WaveNet_"

cae_preds_dir = "preds/test_preds_cae/preds_test_CAE_"
cae_targets_dir = "targets/targets_cae/targets_test_CAE_"
cae_labels_dir = "labels/test_labels_cae/label_list_CAE_"
scenes = (
    "beach",
    "bus",
    "cafe_restaurant",
    "car",
    "city_center",
    "forest_path",
    "grocery_store",
    "home",
    "library",
    "metro_station",
    "office",
    "park",
    "residential_area",
    "train",
    "tram"
)

In [3]:
def results(scene_name, preds_dir, targets_dir, labels_dir):
    preds = np.load("{}{}.npy".format(preds_dir,scene_name))
    preds = np.squeeze(preds)
    targets =np.load("{}{}.npy".format(targets_dir,scene_name))
    targets = np.squeeze(targets)
    labels =np.load("{}{}.npy".format(labels_dir,scene_name))
    return preds, targets, labels

In [4]:
print('WaveNet Results:')
wavenet_df_list = []
for scene in scenes:
    wn_preds, wn_targets, wn_labels = results(scene,wavenet_preds_dir, wavenet_targets_dir, wavenet_labels_dir )
    wn_SE = (wn_preds - wn_targets) ** 2
    wn_mse_col = wn_SE.mean(axis=0)
    wn_mse = wn_SE.mean(axis=1)
    wn_fpr, wn_tpr, wn_thresholds = metrics.roc_curve(wn_labels,wn_mse)
    wn_roc_auc_mse = metrics.auc(wn_fpr, wn_tpr)
    df = pd.DataFrame()
    df['mse'] =  wn_mse
    df['labels'] = wn_labels
    wavenet_df_list.append(df)
    print('{:.2f} {} ({})'.format(wn_roc_auc_mse, scene, wn_roc_auc_mse))

WaveNet Results:
0.72 beach (0.7249212353262138)
0.83 bus (0.8287052631578948)
0.76 cafe_restaurant (0.7561466229663864)
0.82 car (0.8217299387792277)
0.82 city_center (0.8236671102689241)
0.72 forest_path (0.7175636189877798)
0.77 grocery_store (0.7716451089573134)
0.69 home (0.6905454425988069)
0.67 library (0.6662959154254813)
0.79 metro_station (0.790010652017652)
0.78 office (0.7841212117390216)
0.80 park (0.8038107366043231)
0.78 residential_area (0.777699619439862)
0.84 train (0.8382276824461414)
0.87 tram (0.8695021100364341)


In [5]:
cae_df_list = []

print('CAE Results:')
for scene in scenes:
    cae_preds, cae_targets, cae_labels = results(scene,cae_preds_dir, cae_targets_dir, cae_labels_dir )
    cae_SE = (cae_preds - cae_targets) ** 2
    cae_mse = cae_SE.mean(axis=1)
    cae_fpr, cae_tpr, cae_thresholds = metrics.roc_curve(cae_labels,cae_mse)
    cae_roc_auc_mse = metrics.auc(cae_fpr, cae_tpr)
    df_cae = pd.DataFrame()
    df_cae['mse'] =  cae_mse
    df_cae['labels'] = cae_labels
    cae_df_list.append(df_cae)
    print('{:.2f} {} ({})'.format(cae_roc_auc_mse, scene, cae_roc_auc_mse))

CAE Results:
0.69 beach (0.6942539539938926)
0.79 bus (0.7929294414196411)
0.69 cafe_restaurant (0.6925009523016172)
0.79 car (0.7896839815949859)
0.75 city_center (0.751457434700052)
0.65 forest_path (0.6545196343519264)
0.71 grocery_store (0.7050269513862195)
0.69 home (0.6898417472535412)
0.59 library (0.586443300797961)
0.74 metro_station (0.7429961169649938)
0.78 office (0.7796394913934745)
0.70 park (0.6985115740822218)
0.73 residential_area (0.7343311785037272)
0.82 train (0.8230979782799646)
0.80 tram (0.8045354121379408)


In [6]:
dfs = [wavenet_df_list, cae_df_list]

In [7]:
%%R -i  dfs

library(pROC)
wavenet_df_list<-dfs[[1]]
cae_df_list<-dfs[[2]]
for(i in 1:15) {
    roc_1<-roc(wavenet_df_list[[i]]$labels, wavenet_df_list[[i]]$mse)
    roc_2<-roc(cae_df_list[[i]]$labels, cae_df_list[[i]]$mse)
    print(ci.auc(roc_1))
    print(ci.auc(roc_2))
    #print(roc.test(roc_1, roc_2, method=c("delong")))
}

R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Attaching package: ‘pROC’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, smooth, var


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



95% CI: 0.7033-0.7465 (DeLong)
95% CI: 0.6731-0.7154 (DeLong)


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



95% CI: 0.8092-0.8482 (DeLong)
95% CI: 0.7728-0.8131 (DeLong)


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



...

95% CI: 0.756-0.7994 (DeLong)
95% CI: 0.7128-0.7559 (DeLong)


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



95% CI: 0.8178-0.8586 (DeLong)
95% CI: 0.8031-0.8431 (DeLong)


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



95% CI: 0.852-0.887 (DeLong)
95% CI: 0.7865-0.8226 (DeLong)


In [8]:
wavenet_df_pr_list = []
print('WaveNet Results:')
for scene in scenes:
    wn_preds, wn_targets, wn_labels = results(scene,wavenet_preds_dir, wavenet_targets_dir, wavenet_labels_dir )
    wn_SE = (wn_preds - wn_targets) ** 2
    wn_mse_col = wn_SE.mean(axis=0)
    wn_mse = wn_SE.mean(axis=1)
    wn_precision, wn_recall, _ = metrics.precision_recall_curve(wn_labels,wn_mse)
    wn_pr_auc_mse = metrics.auc(wn_recall, wn_precision)
    df_pr = pd.DataFrame()
    df_pr['mse'] =  wn_mse
    df_pr['labels'] = wn_labels
    wavenet_df_pr_list.append(df_pr)
    print('{:.2f} {} ({})'.format(wn_pr_auc_mse, scene, wn_pr_auc_mse))

WaveNet Results:
0.29 beach (0.29379025535415404)
0.48 bus (0.47598223339761025)
0.24 cafe_restaurant (0.23564432425521645)
0.61 car (0.6059104803314989)
0.32 city_center (0.3155788168980619)
0.09 forest_path (0.08872043440048213)
0.24 grocery_store (0.23889167864322527)
0.07 home (0.0734346613481804)
0.18 library (0.17774899063906918)
0.14 metro_station (0.13611963056317913)
0.21 office (0.20684201131174249)
0.46 park (0.45607483271930704)
0.22 residential_area (0.22124409857106883)
0.46 train (0.459781283129454)
0.56 tram (0.563348983744147)


In [9]:
cae_df_pr_list = []
print('CAE Results:')
for scene in scenes:
    cae_preds, cae_targets, cae_labels = results(scene,cae_preds_dir, cae_targets_dir, cae_labels_dir )
    cae_SE = (cae_preds - cae_targets) ** 2
    cae_mse = cae_SE.mean(axis=1)
    cae_precision, cae_recall, _ = metrics.precision_recall_curve(cae_labels,cae_mse)
    cae_pr_auc_mse = metrics.auc(cae_recall, cae_precision)
    df_pr_cae = pd.DataFrame()
    df_pr_cae['mse'] =  cae_mse
    df_pr_cae['labels'] = cae_labels
    cae_df_pr_list.append(df_pr_cae)
    print('{:.2f} {} ({})'.format(cae_pr_auc_mse, scene, cae_pr_auc_mse))

CAE Results:
0.12 beach (0.11714791518898612)
0.38 bus (0.38061454765119745)
0.08 cafe_restaurant (0.07978159613958724)
0.47 car (0.46540222073680715)
0.13 city_center (0.12912540372694345)
0.06 forest_path (0.061924587241173515)
0.10 grocery_store (0.09878014991818483)
0.05 home (0.04615333908956339)
0.03 library (0.025959896021388568)
0.06 metro_station (0.0611757922176793)
0.15 office (0.14997070729091153)
0.06 park (0.05959972071777134)
0.05 residential_area (0.05250776384662553)
0.37 train (0.36687000095952677)
0.39 tram (0.3872904371435507)


In [22]:
from sklearn.utils import resample
import scipy.stats as st
def ci_calc(df):
    sample_auc = []
    for i in range(2000):
        df_boot = resample(df, replace=True, n_samples=len(df))
        precision, recall, _ = metrics.precision_recall_curve(df_boot.labels,df_boot.mse)
        pr_auc_mse = metrics.auc(recall, precision)
        sample_auc.append(pr_auc_mse)

    return st.norm.interval(alpha=0.95, loc=np.mean(sample_auc), scale=st.sem(sample_auc))

for i in range(len(scenes)):
    df_wn_i = wavenet_df_pr_list[i]
    wn_precision, wn_recall, _ = metrics.precision_recall_curve(df_wn_i.labels, df_wn_i.mse)
    wn_pr_auc_mse = metrics.auc(wn_recall, wn_precision)
    print("{}, {}, confidence interval {}".format(scenes[i], wn_pr_auc_mse, ci_calc(df_wn_i)))


beach, 0.29379025535415404, confidence interval (0.29317560168127915, 0.2947330323093588)
bus, 0.47598223339761025, confidence interval (0.475165777778346, 0.4768180155552121)
cafe_restaurant, 0.23564432425521645, confidence interval (0.2356359625636611, 0.23707724421426057)
car, 0.6059104803314989, confidence interval (0.6054881630472997, 0.6071591235653107)
city_center, 0.3155788168980619, confidence interval (0.31538536740721707, 0.31702628822643825)
forest_path, 0.08872043440048213, confidence interval (0.08843804133828642, 0.08935847700009647)
grocery_store, 0.23889167864322527, confidence interval (0.23821244738951097, 0.23998854189369356)
home, 0.0734346613481804, confidence interval (0.07331548564989593, 0.07400095403810661)
library, 0.17774899063906918, confidence interval (0.17762082117296402, 0.17905772023526528)
metro_station, 0.13611963056317913, confidence interval (0.13650232428191822, 0.13761500255931736)
office, 0.20684201131174249, confidence interval (0.2067461445426

In [23]:
for i in range(len(scenes)):
    df_cae_i = cae_df_pr_list[i]
    cae_precision, cae_recall, _ = metrics.precision_recall_curve(df_cae_i.labels, df_cae_i.mse)
    cae_pr_auc_mse = metrics.auc(cae_recall, cae_precision)
    print("{}, {}, confidence interval {}".format(scenes[i], cae_pr_auc_mse, ci_calc(df_cae_i)))


beach, 0.11714791518898612, confidence interval (0.11668453947777523, 0.1177100138291048)
bus, 0.38061454765119745, confidence interval (0.3790191222592182, 0.3806519321780527)
cafe_restaurant, 0.07978159613958724, confidence interval (0.0796558411181238, 0.08031919634036377)
car, 0.46540222073680715, confidence interval (0.46391641805092, 0.46565214546072564)
city_center, 0.12912540372694345, confidence interval (0.12851034711065767, 0.1296074515648582)
forest_path, 0.061924587241173515, confidence interval (0.06190200470150765, 0.06261800444226524)
grocery_store, 0.09878014991818483, confidence interval (0.09913621676530199, 0.10028755070304254)
home, 0.04615333908956339, confidence interval (0.04603750882983319, 0.04627930149449154)
library, 0.025959896021388568, confidence interval (0.026156017908079258, 0.026325819362201793)
metro_station, 0.0611757922176793, confidence interval (0.06110664319310277, 0.06155820429153977)
office, 0.14997070729091153, confidence interval (0.14996890